<a href="https://colab.research.google.com/github/jgoodrich77/deep-text-corrector/blob/master/TextCorrector_jg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Start with moving to directory and then setting up environment

In [0]:
!cd ~/

Clone repository into colab

In [19]:
!git clone https://github.com/jgoodrich77/deep-text-corrector.git

Cloning into 'deep-text-corrector'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 130 (delta 2), reused 0 (delta 0), pack-reused 122
Receiving objects: 100% (130/130), 297.15 KiB | 9.90 MiB/s, done.
Resolving deltas: 100% (70/70), done.


Move into the directory

In [0]:
!cd /content/deep-text-corrector

Install requirements

In [24]:
!pip3 install -r requirements.txt

Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [0]:
!rm -rf /content/deep-text-corrector

In [0]:
!ls -lta

In [22]:
!pwd

/content


In [7]:
from __future__ import print_function

import os
import time
import numpy as np
import tensorflow as tf
import pandas as pd
from collections import defaultdict

from sklearn.metrics import roc_auc_score, accuracy_score
import nltk

from correct_text import train, decode, decode_sentence, evaluate_accuracy, create_model,\
    get_corrective_tokens, DefaultPTBConfig, DefaultMovieDialogConfig
from text_correcter_data_readers import PTBDataReader, MovieDialogReader

%matplotlib inline

ImportError: ignored

In [0]:
root_data_path = "/Users/atpaino/data/textcorrecter/dialog_corpus"
train_path = os.path.join(root_data_path, "movie_lines.txt")
val_path = os.path.join(root_data_path, "cleaned_dialog_val.txt")
test_path = os.path.join(root_data_path, "cleaned_dialog_test.txt")
model_path = os.path.join(root_data_path, "dialog_correcter_model_testnltk")
config = DefaultMovieDialogConfig()

## Train

In [0]:
data_reader = MovieDialogReader(config, train_path)

In [0]:
train(data_reader, train_path, val_path, model_path)

## Decode sentences

In [0]:
data_reader = MovieDialogReader(config, train_path, dropout_prob=0.25, replacement_prob=0.25, dataset_copies=1)

In [0]:
corrective_tokens = get_corrective_tokens(data_reader, train_path)

In [0]:
import pickle
with open(os.path.join(root_data_path, "corrective_tokens.pickle"), "w") as f:
    pickle.dump(corrective_tokens, f)

In [0]:
import pickle
with open(os.path.join(root_data_path, "token_to_id.pickle"), "w") as f:
    pickle.dump(data_reader.token_to_id, f)

In [0]:
sess = tf.InteractiveSession()
model = create_model(sess, True, model_path, config=config)

In [0]:
# Test a sample from the test dataset.
decoded = decode_sentence(sess, model, data_reader, "you must have girlfriend", corrective_tokens=corrective_tokens)

In [0]:
decoded

In [0]:
decoded = decode_sentence(sess, model, data_reader,
                          "did n't you say that they 're going to develop this revolutionary new thing ...",
                          corrective_tokens=corrective_tokens)

In [0]:
decode_sentence(sess, model, data_reader, "kvothe went to market", corrective_tokens=corrective_tokens, verbose=False)

In [0]:
decode_sentence(sess, model, data_reader, "blablahblah and bladdddd went to market", corrective_tokens=corrective_tokens,
                verbose=False)

In [0]:
decode_sentence(sess, model, data_reader, "do you have book", corrective_tokens=corrective_tokens, verbose=False)

In [0]:
decode_sentence(sess, model, data_reader, "the cardinals did better then the cubs", corrective_tokens=corrective_tokens, verbose=False)

In [0]:
# 4 layers, 40k steps
errors = evaluate_accuracy(sess, model, data_reader, corrective_tokens, test_path)#, max_samples=1000)

In [0]:
# 4 layers, 30k steps
errors = evaluate_accuracy(sess, model, data_reader, corrective_tokens, test_path)#, max_samples=1000)

In [0]:
# 4 layers, 20k steps
errors = evaluate_accuracy(sess, model, data_reader, corrective_tokens, test_path)#, max_samples=1000)

In [0]:
errors = evaluate_accuracy(sess, model, data_reader, corrective_tokens, test_path)#, max_samples=1000)

In [0]:
for decoding, target in errors:
    print("Decoding: " + " ".join(decoding))
    print("Target:   " + " ".join(target) + "\n")